# ◆GBDT(勾配ブースティング木)
***
目的関数を改善するような決定木を設定した木の本数分だけ繰り返す。<br>
二本目以降はその直前のモデルの出力値(予測値)の差に対して学習が行われていくイメージ。

# ◆特徴
***
* 特徴量は数値<br>
    ある特徴量の値の大小で分類していくためすべて数値である必要がある。<br>
    <br>
* 欠損値を扱える<br>
    決定木で欠損は欠損で分けることができるため特徴量に欠損が入ってても問題ない。<br>
<br>
* 変数間の相互作用が反映される<br>
    分岐が繰り返されることにより、変数間の相互作用が反映される<br>
    例)男性かどうかという分岐の後に20歳以上かという分岐があれば男性で20歳以上という様に相互作用が反映<br>
<br>
* 特徴量をスケーリングする必要がない<br>
    それぞれの特徴量の大小関係を見るためスケールは関係ない。<br>
<br>
* カテゴリ変数をOne-Hotに展開する必要がない<br>
    数値である必要があるためlabel encodingはする必要がある。<br>

# ◆主なライブラリ
***
GBDTのアルゴリズムを用いているライブラリには以下のものがある。

* xgboost
* lightgbm
* catboost

## xgboost

### 使い方の例

In [ ]:
import xgboost as xgb
from sklearn.metrcs import log_loss

# 特徴量と目的変数をxgboostのデータ構造に変換する。
dtrain = xgb.DMatrix(tr_x, label=tr_y)
dvalid = xgb.DMatrix(va_x, label=va_y)
dtest = xgb.DMatrix(test_x)

# パラメータ設定
params = {'objective': 'binary:logistic', 'silent':1, 'random_state':71}
num_round =50

# 学習の実行
# バリデーションデータもモデルに渡してスコアの変化のモニタリング
watch_list = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(params, dtrain, num_round, evals=watchlist)

# バリデーションデータでのスコア確認
va_pred = model.predict(dvalid)
score = log_loss(va_y, va_pred)
print(f'logloss: {score:.4f}')

# 予測
pred = model.predict(dtest)

### その他の設定

* booster指定<br>
    デフォルトでは'gbtree'となっている。通常ではこのままでいい。<br>
    'dart'を指定することによりDARTというアルゴリズムを使用した正則化項を適用する。<br>
<br>
* 目的関数<br>
    目的関数を設定するにはパラメータ'objective'を以下の様に設定する。
    * reg:squarederror<br>
        回帰の場合に設定することで平均二乗誤差を最小化するように学習する。
    * binary:ligistic<br>
        二値分類の場合に設定することでloglossを最小化するように学習を行う。
    * multi:softprob<br>
        マルチクラス分類の場合に設定することでmulti-class loglossを最小化するように学習を行う。
<br>
* バリデーション<br>
    trainメソッドのevalsパラメータに学習データとバリデーションデータを渡すことで決定木を追加するごとにバリデーションデータへのスコアを出力する

### アーリーストッピング
trainメソッドのearly_stopping_roundsパラメータを指定することで、アーリーストッピングを行うことができる。<br>

※アーリーストッピングを行う場合、予測時にntree_limitパラメータを設定しないと最適ではなく学習が止まったところまでで予測を行ってしまう。